### 크롤링 도중 ip 차단을 당했고 실시간으로 ip를 랜덤하게 변환시키는 방법을 찾음<br> 그러나 steamspy에서 제공하는 api로 id와 name을 찾음<br> 시간상의 이유나 모든 상황을 볼때 새로 steam spy에서 크롤링하는것이 더 좋은 판단이라고 생각되어 크롤링을 새로 시작

# 크롤링

In [6]:
import time
import pandas as pd
import requests
import pymysql
import json
from bs4 import BeautifulSoup
from sqlalchemy import create_engine
from selenium import webdriver

In [7]:
host_name = "127.0.0.1"
username = "root"
password = ""
database_name = "steam_test"

In [8]:
db = pymysql.connect(
    host = host_name,
    port=3306,
    user=username,
    passwd=password,
    db=database_name,
    charset='utf8'
)

## 테이블 생성

In [4]:
start_time = time.time()
#-----------------------------------------------------------------------------------------
chromedriver = '/home/nincastle/chromedriver/chromedriver' # ubuntu
# chromedriver = 'D:\chromedriver_win32/chromedriver' # windows
tags_genre_query = []

def name_replace(name):
    name = name.replace("&","and").replace(".","dot")\
    .replace("Mod","Mod_").replace("'","").replace("-","_").replace("Linear","Linear_")\
    .replace("Action","Action_").replace("Foreign","Foreign_").replace("All","All_")
    return name

def driver_chrome(url, xpath_):
    driver_tags = webdriver.Chrome(chromedriver)
    driver_tags.get(url)
    print(type(xpath_))
    tags = driver_tags.find_element_by_xpath(xpath_)
    t_g = tags.find_elements_by_tag_name('a')
    driver_tags.quit
    return t_g
# 테그 변수 이름 크롤링
cro_tag = driver_chrome("https://steamdb.info/tags/","//div[@class='row list']")
for t in cro_tag:
    t = t.text.replace(" ","_")   
    tags_genre_query.append("tag_"+t+' INT(1)')


# 장르 변수 이름 크롤링
cro_gen = driver_chrome("https://steamdb.info/genres/","//div[@class='row']")
for g in cro_gen:
    g = g.text.replace(" ","_")
    tags_genre_query.append("genre_"+g+' INT(1)')

# driver_tags.quit
tag_genre_join = (", \n".join(tags_genre_query))

# SQL문 필터링
tag_genre_join = name_replace(tag_genre_join)
try: 
    cursor = db.cursor()
    SQL_QUERY = """
    # DROP DATABASE IF EXISTS 테이블이름;
    # CREATE DATABASE 테이블이름 DEFAULT CHARSET=utf8 COLLATE=utf8_bin;

    USE steam_test;
    DROP TABLE IF EXISTS steam_test.data_merge;
    CREATE TABLE steam_test.data_merge (
        appid int(7),
        name varchar(100),
        owners int(10),
        owners_variance int(7),
        negative int(7),
        positive int(6),
        score_rank int(3),
        ccu int(6),
        players_2weeks int(8),
        players_forever_variance int(7),
        players_2weeks_variance int(7),
        userscore int(2),
        median_forever int(3),
        players_forever int(7),
        price int(7),
        average_2weeks int(5),
        developer varchar(300),
        median_2weeks int(4),
        publisher varchar(100),
        average_forever int(6),
        tag_Dystopian_ int(1),
        tag_Parody_ int(1),
        tag_VR_Only int(1),
        """+tag_genre_join+""",
        PRIMARY KEY(appid)
        ) DEFAULT CHARSET=utf8 COLLATE=utf8_bin
    """

    create_sql=("".join(SQL_QUERY))
    print(create_sql)
    cursor.execute(create_sql)
    db.commit()
finally:
    print("ok!")
    pass

#----------------------------------------------------------------------------------------    
end_time = time.time()
print("{} 초(처리시간)".format(end_time-start_time))

### 초기 ID, name 데이터

In [5]:
start_time = time.time()
#-----------------------------------------------------------------------------------------
with open("request_all.txt","r") as file:
    f = file.read()
    data = json.loads(f)
    for k,v in data.items():
        key = []
        value = []
        for k2,v2 in v.items():
            print(k2," : ",v2)
            if k2 == "score_rank":
                if v2 == "":
                    v2 = 0
            elif k2 == "developer" or k2 =="publisher":
                if v2 =="":
                    v2 = "null"
            elif k2 == "price":
                if v2 == "":
                    v2 == 999999                    
            key.append(k2)
            value.append(v2)
            
        key = tuple(key)
        key = ", ".join(key)
        value = tuple(value)
        sql ="""
            INSERT INTO steam_test.steamtable2 ({}) 
            VALUES 
            {};
            """.format(key, value)
        print(sql)
        cursor.execute(sql)
        db.commit()
#----------------------------------------------------------------------------------------    
end_time = time.time()
print("{} 초(처리시간)".format(end_time-start_time))

### 최종데이터 (크롤링 기간 2018-01-16~2018-01-19)

In [9]:
start_time = time.time()
#-----------------------------------------------------------------------------------------

def name_replace(name):
    name = name.replace("&","and").replace(".","dot")\
    .replace("Mod","Mod_").replace("'","").replace("-","_").replace("Linear","Linear_")\
    .replace("Action","Action_").replace("Foreign","Foreign_").replace("All","All_")
    return name

def select_number(start, last):
    get_data = """
    # select appid, name from steamtable2 LIMIT 2
    SELECT appid, name
    FROM steamtable2
    WHERE
        appid BETWEEN {} AND {};
    """.format(start, last)
    
    cursor = db.cursor()
    cursor.execute(get_data)
    row_names = cursor.fetchall()

    count = 0
    for s_num in row_names:
        print(s_num[0],s_num[1],count)
        count += 1  
        column_str=[]
        row_str=[]
        
        steamspy_url =('https://steamspy.com/api.php?request=appdetails&appid={}'.format(s_num[0]))
        appdetails_response = requests.get(steamspy_url)
        data = appdetails_response.json()
        
        for data_k, data_v in data.items():
            data_k = name_replace(data_k).replace(" ","_")  
            if data_k == 'tags':
                if data['tags'] == []:
                    pass
                else:
                    for tag_k ,tag_v in data['tags'].items():
                        tag_k = name_replace(tag_k).replace(" ","_")
                        column_str.append(("tag_{}").format(tag_k))
                        row_str.append(1)
                        
            elif data_k == "score_rank":
                if data_k == "":
                    column_str.append(data_k)
                    row_str.append(0)
                    
            elif data_k == "developer" or data_k =="publisher":
                if data_v =="":
                    column_str.append(data_k)
                    row_str.append("null")
                    
            elif data_k == "price":
                if data_v == "":
                    column_str.append(data_k)
                    row_str.append(999999)
                    
            else:
                column_str.append(data_k)
                row_str.append(data_v)
        try:
            cursor = db.cursor()
            sql = ("INSERT INTO steam_test.data_merge ("+(",".join(column_str)+") VALUES {}"\
                                                          .format(tuple(row_str))))
            cursor.execute(sql)
            db.commit()
        finally:
            pass    
#----------------------------------------------------------------------------------------


select_number(10, 999999)
#----------------------------------------------------------------------------------------    
end_time = time.time()
print("{} 초(처리시간)".format(end_time-start_time))

# 데이터 정제

추가
* data_merge + steamtable2

In [57]:
copy_table = """
DROP TABLE IF EXISTS steam_test.merge_steam;
create table steam_test.merge_steam like steam_test.data_merge
"""
insert_data_merge = """
INSERT INTO steam_test.merge_steam 
SELECT * FROM steam_test.data_merge;
"""
delet_publisher = """
update merge_steam
set publisher = null
where publisher is not null;
"""
delet_developer = """
update merge_steam
set developer = null
where developer is not null;
"""

In [58]:
start_time = time.time()
#-----------------------------------------------------------------------------------------
cursor = db.cursor()
cursor.execute(copy_table)
cursor.execute(insert_data_merge)
cursor.execute(delet_developer)
cursor.execute(delet_publisher)
db.commit()
#----------------------------------------------------------------------------------------    
end_time = time.time()
print("{} 초(처리시간)".format(end_time-start_time))

3.707911729812622 초(처리시간)


In [ ]:
start_time = time.time()
#-----------------------------------------------------------------------------------------
test_steamtable2 ="""
SELECT appid, score_rank, price, developer, publisher FROM steamtable2
"""
cursor = db.cursor()
cursor.execute(test_steamtable2)
a_data = cursor.fetchall()
for i in a_data:
#     print("appid : {}, score_rank: {}, price: {}, developer: {}, publisher:{}".format(i[0], i[1], i[2], i[3], i[4]))
    try:
        update_data = """
        update merge_steam
        set score_rank = {}, price = {}, developer = "{}", publisher = "{}"
        where appid = {};
        """.format(i[1], i[2], i[3], i[4], i[0])
        print(update_data)
        cursor.execute(update_data)
    #     print(update_data)
    except:
        update_data = """
        update merge_steam
        set score_rank = {}, price = {}, developer = '{}', publisher = '{}'
        where appid = {};
        """.format(i[1], i[2], i[3], i[4], i[0])
        print(update_data)
        cursor.execute(update_data)
            
db.commit()
#----------------------------------------------------------------------------------------    
end_time = time.time()
print("{} 초(처리시간)".format(end_time-start_time))

teg, genre null to 0

In [ ]:
start_time = time.time()
#-----------------------------------------------------------------------------------------
# 변수명
c_name = """
SELECT COLUMN_NAME
FROM information_schema.columns 
WHERE 
    table_schema = 'steam_test' 
    AND 
    table_name = 'merge_steam';
"""
cursor = db.cursor()
cursor.execute(c_name)

# tag range 20 ~ 362
# genre range 363 ~ 387

tg_list = []
for a in cursor:
    a = str(a)
    filter_a = a.replace("'","").replace(",","")
    tg_list.append(filter_a)

for a in tg_list[21:387]:
    a = a.replace("(","").replace(")","")
    sql_a = "update steam_test.merge_steam set {} = IFNULL({}, 0); ".format(a, a)
    print(sql_a)
    cursor.execute(sql_a)

db.commit()
cursor.close()
#----------------------------------------------------------------------------------------    
end_time = time.time()
print("{} 초(처리시간)".format(end_time-start_time))